In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:

zip_ref = zipfile.ZipFile("/content/drive/My Drive/10kArticles.zip", 'r')
zip_ref.extractall("/tmp/")
zip_ref.close()

articleLoc = '/tmp/ArticleFetching'

In [ ]:
pip install transformers

     |████████████████████████████████| 4.2 MB 3.1 MB/s 
     |████████████████████████████████| 6.6 MB 40.7 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 596 kB 34.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import AutoTokenizer, AutoConfig, TFAutoModel
import os

In [ ]:
 
MODEL_NAME = 'allenai/longformer-base-4096'
MAX_LEN = 500

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('model')

config = AutoConfig.from_pretrained(MODEL_NAME) 
config.save_pretrained('model')

backbone = TFAutoModel.from_pretrained(MODEL_NAME, config=config)
backbone.save_pretrained('model')
DOWNLOADED_MODEL_PATH = '/content/model'

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/729M [00:00<?, ?B/s]

Some layers from the model checkpoint at allenai/longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at allenai/longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.


In [ ]:
def build_model():
    
    tokens = tf.keras.layers.Input(shape=(MAX_LEN,), name = 'tokens', dtype=tf.int32)
    attention = tf.keras.layers.Input(shape=(MAX_LEN,), name = 'attention', dtype=tf.int32)
    
    config = AutoConfig.from_pretrained(DOWNLOADED_MODEL_PATH+'/config.json') 
    backbone = TFAutoModel.from_pretrained(DOWNLOADED_MODEL_PATH+'/tf_model.h5', config=config)
    
    x = backbone(tokens, attention_mask=attention)
    
    model = tf.keras.Model(inputs=[tokens,attention], outputs=x)
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 1e-4),
                  loss = [tf.keras.losses.CategoricalCrossentropy()],
                  metrics = [tf.keras.metrics.CategoricalAccuracy()])
    
    return model

In [ ]:
model = build_model()

All model checkpoint layers were used when initializing TFLongformerModel.

All the layers of TFLongformerModel were initialized from the model checkpoint at /content/model/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:
from scipy.spatial import distance
# Straipsnis destytojo:
lecturerArticleLoc = '/tmp/test.txt'
f = open(lecturerArticleLoc, "r", encoding='cp1252')
text = f.read()
partitioned_string = text.partition('.')
before_first_period = partitioned_string[0]
print(before_first_period)
tokens = tokenizer.encode_plus(before_first_period, max_length=MAX_LEN, padding='max_length',
                            truncation=True, return_offsets_mapping=True)
test_tokens = np.zeros((1,MAX_LEN), dtype='int32')
test_attention = np.zeros((1,MAX_LEN), dtype='int32')
test_tokens[0,] = tokens['input_ids']
test_attention[0,] = tokens['attention_mask']
prediction = model.predict([test_tokens, test_attention])
# A = np.array([prediction.last_hidden_state[0]], dtype=object)
# lecturerArticleFeatures = A.flatten()
lecturerArticleFeatures = cls_pooling(prediction)
# print(lecturerArticleFeatures)
# result = 1 - distance.cosine(feature_array[0], lecturerArticleFeatures)
# print(result)


Fourth-dose Covid booster vaccines increase protection against Covid-19, particularly in people aged over 70, according to a UK trial


In [ ]:
import pandas as pd
import csv  
from scipy import spatial
dictionary = dict()

import pandas as pd
featureLoc = '/content/drive/My Drive/featuresOneSentence.csv'
metadata = pd.read_csv(featureLoc)
for index_num,row in metadata.iterrows():
  strLen = len(row[1])
  numArrStr = row[1][1:strLen-1]
  numStrArr = numArrStr.split()
  numbers = list(map(float, numStrArr))
  result = 1 - spatial.distance.cosine(lecturerArticleFeatures, numbers)
  dictionary[row[0]] = result

finalDict = {k: v for k, v in sorted(dictionary.items(), key=lambda item: item[1], reverse=True)}


In [ ]:
counter = 0
for k, v in finalDict.items():
  if counter > 5:
    break
  print(str(k),v)
  f = open(articleLoc+'/'+k, "r",  encoding='cp1252')
  text = f.read()
  partitioned_string = text.partition('.')
  before_first_period = partitioned_string[0]
  print(before_first_period)
  print('---------------------------------')
  counter = counter + 1 

622.txt 0.9984119475325639
Studies show that the benefits of COVID-19 vaccination in pregnant people outweigh any known or potential risks, according to the CDC
---------------------------------
448.txt 0.998168525299117
• In England, the rate of a positive COVID-19 test is higher in vaccinated people aged 30 and older, compared with unvaccinated people
---------------------------------
776.txt 0.9977084602971983
Scientific evidence and studies show that unvaccinated people are more than five times more likely than vaccinated people to catch COVID-19, and over 10 times more likely to be hospitalized or die from COVID-19
---------------------------------
641.txt 0.9976972093099676
Blood clots and heart problems linked to the COVID-19 vaccines are rare, but the virus itself continues to cause deaths and hospitalizations
---------------------------------
1156.txt 0.9975912445922396
The delta variant has lowered the effectiveness of the vaccines, although researchers aren't exactly sure to